### Import Libraries and Dataset

In [2]:
import pandas as pd
import numpy as np
from decimal import Decimal, ROUND_DOWN

In [3]:
# Load historical funding rate data
df = pd.read_csv("dataset/binance_btc_usdt_1_year_data_8h.csv")

### Parameters

In [4]:
# Parameters
min_funding_rate = 0.0001  # Minimum funding rate to consider arbitrage
max_funding_rate = 0.01    # Maximum funding rate to prevent outlier trades
trade_size = 0.01          # Base BTC per trade
max_trade_size = 0.05      # Maximum position size
max_open_positions = 3     # Maximum number of simultaneous open positions
max_drawdown_pct = 0.05    # Maximum allowed drawdown (5%)
stop_loss_pct = 0.02       # Stop loss percentage (2%)

# RSI Parameters
rsi_period = 14            # Period for RSI calculation
rsi_oversold = 30          # Oversold threshold
rsi_overbought = 70        # Overbought threshold

# Risk-free rate (for Sharpe ratio calculation, typically treasury yield)
risk_free_rate = 0.02      # 2% annual risk-free rate

# Risk management variables
account_balance = 10000    # Starting account balance in USD
open_positions = []        # Track open positions
total_profit = 0           # Track total profit
max_balance = account_balance  # Track maximum balance for drawdown calculation
daily_returns = []         # List to store daily returns for Sharpe ratio calculation


### Functions
- calculate_rsi: Calculates the Relative Strength Index technical indicator
- calculate_sharpe_ratio: Evaluates risk-adjusted performance
- check_arbitrage_opportunity: Identifies potential arbitrage opportunities
- calculate_position_size: Sizes positions based on funding rate magnitude
- open_hedge_position: Executes the opening of hedge positions in spot and futures markets
- close_positions: Closes existing positions
- calculate_total_profit: Calculates the total profit from an arbitrage position
- check_stop_loss: Determines if a stop loss has been triggered
- funding_arbitrage_bot: The main function that runs the entire strategy backtest

In [8]:
# Function to calculate RSI
def calculate_rsi(data, period=14):
    # Calculate price changes
    delta = data.diff()
    
    # Separate gains and losses
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    # Calculate average gain and loss
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    # Calculate relative strength
    rs = avg_gain / avg_loss
    
    # Calculate RSI
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# Function to calculate Sharpe ratio
def calculate_sharpe_ratio(returns, risk_free_rate, trading_days=365):
    """
    Calculate the annualized Sharpe ratio.
    
    Parameters:
    - returns: List of daily returns
    - risk_free_rate: Annual risk-free rate (e.g., 0.02 for 2%)
    - trading_days: Number of trading days in a year
    
    Returns:
    - Sharpe ratio
    """
    if not returns or len(returns) < 2:
        return 0
    
    # Convert annual risk-free rate to daily
    daily_risk_free = (1 + risk_free_rate) ** (1 / trading_days) - 1
    
    # Calculate excess returns
    excess_returns = [r - daily_risk_free for r in returns]
    
    # Calculate mean and standard deviation of excess returns
    mean_excess_return = np.mean(excess_returns)
    std_excess_return = np.std(excess_returns, ddof=1)  # Sample standard deviation
    
    if std_excess_return == 0:
        return 0
    
    # Daily Sharpe ratio
    daily_sharpe = mean_excess_return / std_excess_return
    
    # Annualized Sharpe ratio
    annualized_sharpe = daily_sharpe * np.sqrt(trading_days)
    
    return annualized_sharpe

# Function to round trade size to 5 decimal places
def round_trade_size(size):
    return float(Decimal(str(size)).quantize(Decimal('0.00001'), rounding=ROUND_DOWN))

# Function to check if arbitrage opportunity exists
def check_arbitrage_opportunity(df, min_funding_rate, max_funding_rate):
    # Filter out opportunities that are too small or suspiciously large
    return df[(abs(df['funding_rate']) >= min_funding_rate) & 
            (abs(df['funding_rate']) <= max_funding_rate)]

# Function to size position based on funding rate magnitude
def calculate_position_size(funding_rate, trade_size, max_trade_size):
    # Scale position size based on funding rate magnitude (larger funding = larger position)
    # But never exceed max_trade_size
    scaled_size = trade_size * (abs(funding_rate) / min_funding_rate)
    # Round to 5 decimal places
    return round_trade_size(min(scaled_size, max_trade_size))

# Function to determine hedge positions
def open_hedge_position(trade_size, funding_rate, mark_price):
    if funding_rate > 0: 
        side_spot = 'buy'
        side_futures = 'sell'
    else:
        side_spot = 'sell'
        side_futures = 'buy'
    
    position_value = trade_size * mark_price
    print(f"Opened hedge positions: {side_futures} futures and {side_spot} spot, Size: {trade_size} BTC (${position_value:.2f})")
    return side_spot, side_futures

# Function to close positions
def close_positions(trade_size, funding_rate, mark_price):
    if funding_rate > 0:
        side_spot = 'sell'
        side_futures = 'buy'
    else:
        side_spot = 'buy'
        side_futures = 'sell'
    
    position_value = trade_size * mark_price
    print(f"Closed hedge positions: {side_spot} spot and {side_futures} futures, Size: {trade_size} BTC (${position_value:.2f})")
    return side_spot, side_futures

# Function to calculate total profit
def calculate_total_profit(trade_size, funding_rate, mark_price, entry_spot, close_spot, entry_futures, close_futures):
    """
    Calculates total funding arbitrage profit:
    - Funding fee profit
    - Spot position profit/loss
    - Futures position profit/loss
    """
    # Funding Profit (8-hour funding period)
    funding_profit = trade_size * funding_rate * mark_price

    # Determine trade direction based on funding rate
    if funding_rate > 0:
        spot_side_multiplier = 1     # Buying spot (long)
        futures_side_multiplier = -1  # Shorting futures
    else:
        spot_side_multiplier = -1     # Selling spot (short)
        futures_side_multiplier = 1   # Longing futures

    # Spot Profit/Loss
    spot_pnl = trade_size * (close_spot - entry_spot) * spot_side_multiplier

    # Futures Profit/Loss
    futures_pnl = trade_size * (entry_futures - close_futures) * futures_side_multiplier

    # Total Profit
    total_profit = funding_profit + spot_pnl + futures_pnl

    return total_profit, funding_profit, spot_pnl, futures_pnl

# Check if stop loss is triggered
def check_stop_loss(position, current_spot, current_futures, stop_loss_pct):
    funding_rate = position['funding_rate']
    entry_spot = position['entry_spot']
    entry_futures = position['entry_futures']
    trade_size = position['trade_size']
    
    # Calculate current PnL (excluding funding, which is already locked in)
    if funding_rate > 0:  # Long spot, short futures
        spot_pnl = trade_size * (current_spot - entry_spot)
        futures_pnl = trade_size * (entry_futures - current_futures)
    else:  # Short spot, long futures
        spot_pnl = trade_size * (entry_spot - current_spot)
        futures_pnl = trade_size * (current_futures - entry_futures)
    
    current_pnl = spot_pnl + futures_pnl
    position_value = trade_size * entry_spot
    
    # Check if loss exceeds stop loss threshold
    if current_pnl < -position_value * stop_loss_pct:
        return True
    return False

# Function to run the funding arbitrage bot with risk management and RSI
def funding_arbitrage_bot(df, trade_size, min_funding_rate, max_funding_rate, max_open_positions, stop_loss_pct, max_drawdown_pct, rsi_period, rsi_oversold, rsi_overbought, risk_free_rate):
    global account_balance, open_positions, total_profit, max_balance, daily_returns
    
    results = []
    position_id = 0
    
    # Calculate RSI for both spot and futures prices
    df['spot_rsi'] = calculate_rsi(df['close_spot'], rsi_period)
    df['futures_rsi'] = calculate_rsi(df['close_futures'], rsi_period)
    
    # Add date column for grouping returns by day
    df['date'] = pd.to_datetime(df['timestamp']).dt.date
    
    # Create a daily balance tracker for Sharpe ratio calculation
    daily_balance = {}
    prev_balance = account_balance
    current_date = None
    
    # Filter for arbitrage opportunities
    arbitrage_df = check_arbitrage_opportunity(df, min_funding_rate, max_funding_rate)

    for index, row in df.iterrows():
        # Skip first RSI_PERIOD rows as RSI is not calculated yet
        if index < rsi_period:
            continue
            
        timestamp = row['timestamp']
        current_date = row['date']
        current_spot = row['close_spot']
        current_futures = row['close_futures']
        spot_rsi = row['spot_rsi']
        futures_rsi = row['futures_rsi']
        
        # 1. Check existing positions for stop loss or closing
        positions_to_close = []
        for pos in open_positions:
            # Check if stop loss is triggered
            if check_stop_loss(pos, current_spot, current_futures, stop_loss_pct):
                print(f"\n[Position {pos['id']}] ⚠️ Stop loss triggered! Closing position.")
                positions_to_close.append(pos)
            
            # Check if position has reached its target close time
            elif index >= pos['target_close_index']:
                print(f"\n[Position {pos['id']}] Target close time reached. Closing position.")
                positions_to_close.append(pos)
            
            # Check if RSI signals position exit
            elif (pos['funding_rate'] > 0 and spot_rsi > rsi_overbought) or (pos['funding_rate'] < 0 and spot_rsi < rsi_oversold):
                print(f"\n[Position {pos['id']}] ⚠️ RSI exit signal! Spot RSI: {spot_rsi:.2f}. Closing position.")
                positions_to_close.append(pos)
        
        # Close positions that need closing
        for pos in positions_to_close:
            close_positions(pos['trade_size'], pos['funding_rate'], current_spot)
            
            # Calculate final profit
            profit, funding_profit, spot_pnl, futures_pnl = calculate_total_profit(
                pos['trade_size'], pos['funding_rate'], pos['mark_price'],
                pos['entry_spot'], current_spot, pos['entry_futures'], current_futures
            )
            
            # Update account balance
            account_balance += profit
            total_profit += profit
            max_balance = max(max_balance, account_balance)
            
            # Log results
            results.append([
                timestamp, pos['funding_rate'], pos['trade_size'], 
                pos['entry_spot'], current_spot, pos['entry_futures'], current_futures,
                profit, funding_profit, spot_pnl, futures_pnl, 
                pos['entry_spot_rsi'], spot_rsi, pos['entry_futures_rsi'], futures_rsi
            ])
            
            print(f"Position closed. Profit: ${profit:.2f} (Funding: ${funding_profit:.2f}, Spot: ${spot_pnl:.2f}, Futures: ${futures_pnl:.2f})")
            print(f"RSI at entry - Spot: {pos['entry_spot_rsi']:.2f}, Futures: {pos['entry_futures_rsi']:.2f}")
            print(f"RSI at exit - Spot: {spot_rsi:.2f}, Futures: {futures_rsi:.2f}")
            open_positions.remove(pos)
        
        # 2. Check drawdown - pause trading if exceeded
        current_drawdown = (max_balance - account_balance) / max_balance if max_balance > 0 else 0
        if current_drawdown > max_drawdown_pct:
            print(f"⚠️ Max drawdown exceeded ({current_drawdown:.2%}). Pausing new trades.")
            continue
        
        # 3. Check for new arbitrage opportunity
        funding_rate = row['funding_rate']
        if abs(funding_rate) >= min_funding_rate and abs(funding_rate) <= max_funding_rate:
            
            # Check if we can open new positions
            if len(open_positions) < max_open_positions:
                mark_price = row['close_futures']  # Mark price is usually close to futures price
                entry_spot = row['close_spot']
                entry_futures = row['close_futures']
                
                # Size position based on funding rate magnitude
                adjusted_size = calculate_position_size(funding_rate, trade_size, max_trade_size)
                
                # Calculate capital needed
                position_value = adjusted_size * mark_price
                
                # RSI filter logic for entry
                rsi_entry_condition = False
                
                if funding_rate > 0:  # We'll go long spot, short futures
                    # For long spot: Enter when spot is oversold (low RSI)
                    # For short futures: Enter when futures is overbought (high RSI)
                    rsi_entry_condition = (spot_rsi < rsi_oversold or futures_rsi > rsi_overbought)
                else:  # We'll short spot, long futures
                    # For short spot: Enter when spot is overbought (high RSI)
                    # For long futures: Enter when futures is oversold (low RSI)
                    rsi_entry_condition = (spot_rsi > rsi_overbought or futures_rsi < rsi_oversold)
                
                # Check if we have enough capital (using 50% of available balance per trade) and RSI condition is met
                if position_value <= account_balance * 0.5 and rsi_entry_condition:
                    print(f"\n[New Position {position_id}] Arbitrage Opportunity Found! Funding Rate: {funding_rate}")
                    print(f"RSI - Spot: {spot_rsi:.2f}, Futures: {futures_rsi:.2f}")
                    
                    # Open hedge positions
                    side_spot, side_futures = open_hedge_position(adjusted_size, funding_rate, mark_price)
                    
                    # Target closing at next funding time (8 hours later) or 3 data points if using hourly data
                    target_close_index = index + 3
                    
                    # Record position
                    position = {
                        'id': position_id,
                        'timestamp': timestamp,
                        'funding_rate': funding_rate,
                        'trade_size': adjusted_size,
                        'entry_spot': entry_spot,
                        'entry_futures': entry_futures,
                        'mark_price': mark_price,
                        'open_index': index,
                        'target_close_index': target_close_index,
                        'entry_spot_rsi': spot_rsi,
                        'entry_futures_rsi': futures_rsi
                    }
                    open_positions.append(position)
                    position_id += 1
                elif not rsi_entry_condition:
                    print(f"RSI conditions not met for entry. Spot RSI: {spot_rsi:.2f}, Futures RSI: {futures_rsi:.2f}")
                else:
                    print(f"Insufficient balance for new position. Required: ${position_value:.2f}, Available: ${account_balance:.2f}")
        
        # Track daily balance for returns calculation
        if current_date not in daily_balance:
            # If this is a new day, calculate return from previous day
            if prev_balance > 0 and len(daily_balance) > 0:
                # Calculate daily return
                daily_return = (account_balance / prev_balance) - 1
                daily_returns.append(daily_return)
                
            # Store today's balance
            daily_balance[current_date] = account_balance
            prev_balance = account_balance
    
    # Close any remaining open positions at the end of the backtest
    for pos in open_positions:
        current_spot = df.iloc[-1]['close_spot']
        current_futures = df.iloc[-1]['close_futures']
        spot_rsi = df.iloc[-1]['spot_rsi']
        futures_rsi = df.iloc[-1]['futures_rsi']
        
        close_positions(pos['trade_size'], pos['funding_rate'], current_spot)
        
        # Calculate final profit
        profit, funding_profit, spot_pnl, futures_pnl = calculate_total_profit(
            pos['trade_size'], pos['funding_rate'], pos['mark_price'],
            pos['entry_spot'], current_spot, pos['entry_futures'], current_futures
        )
        
        # Update account balance
        account_balance += profit
        total_profit += profit
        
        # Log results
        results.append([
            df.iloc[-1]['timestamp'], pos['funding_rate'], pos['trade_size'], 
            pos['entry_spot'], current_spot, pos['entry_futures'], current_futures,
            profit, funding_profit, spot_pnl, futures_pnl,
            pos['entry_spot_rsi'], spot_rsi, pos['entry_futures_rsi'], futures_rsi
        ])

    # Final balance for the last day
    if current_date and prev_balance > 0:
        daily_return = (account_balance / prev_balance) - 1
        daily_returns.append(daily_return)

    # Calculate Sharpe ratio
    # Trading days per year depends on your data frequency - for 8h data, it would be 3 times per day * 365
    trading_periods_per_year = (24/8) * 365  # For 8-hour data
    sharpe_ratio = calculate_sharpe_ratio(daily_returns, risk_free_rate, trading_periods_per_year)

    # Save results to CSV
    results_df = pd.DataFrame(results, columns=[
        'timestamp', 'funding_rate', 'trade_size', 'entry_spot', 'close_spot', 
        'entry_futures', 'close_futures', 'total_profit', 'funding_profit', 
        'spot_pnl', 'futures_pnl', 'entry_spot_rsi', 'exit_spot_rsi', 
        'entry_futures_rsi', 'exit_futures_rsi'
    ])
    results_df.to_csv("dataset/funding_arbitrage_results_2.csv", index=False)
    
    # Save daily returns for further analysis
    daily_returns_df = pd.DataFrame({
        'date': list(daily_balance.keys()),
        'balance': list(daily_balance.values())
    })
    daily_returns_df['daily_return'] = daily_returns_df['balance'].pct_change()
    daily_returns_df.to_csv("dataset/daily_returns.csv", index=False)
    
    # Calculate additional performance metrics
    total_trades = len(results_df)
    profitable_trades = len(results_df[results_df['total_profit'] > 0])
    win_rate = profitable_trades / total_trades if total_trades > 0 else 0
    avg_profit = results_df['total_profit'].mean() if total_trades > 0 else 0
    
    # Calculate volatility (standard deviation of returns)
    volatility = np.std(daily_returns, ddof=1) * np.sqrt(trading_periods_per_year) if daily_returns else 0
    
    # Calculate max drawdown
    cumulative_returns = np.cumprod(np.array([1 + r for r in daily_returns]))
    running_max = np.maximum.accumulate(cumulative_returns)
    drawdowns = (cumulative_returns / running_max) - 1
    max_dd = abs(np.min(drawdowns)) if len(drawdowns) > 0 else 0
    
    # Calculate annualized return
    if len(daily_returns) > 0:
        cagr = ((account_balance / 10000) ** (365 / len(daily_balance))) - 1
    else:
        cagr = 0
    
    print("\n=== PERFORMANCE SUMMARY ===")
    print(f"Starting Balance: ${10000:.2f}")
    print(f"Final Balance: ${account_balance:.2f}")
    print(f"Total Profit: ${total_profit:.2f} ({(account_balance/10000 - 1)*100:.2f}%)")
    print(f"Trading Days: {len(daily_balance)}")
    print(f"Annualized Return: {cagr:.2%}")
    print(f"Annualized Volatility: {volatility:.2%}")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"Max Drawdown: {max_dd:.2%}")
    print(f"Total Trades: {total_trades}")
    print(f"Win Rate: {win_rate:.2%}")
    print(f"Average Profit per Trade: ${avg_profit:.2f}")
    
    # RSI effectiveness statistics
    if total_trades > 0:
        avg_entry_spot_rsi = results_df['entry_spot_rsi'].mean()
        avg_exit_spot_rsi = results_df['exit_spot_rsi'].mean()
        print(f"\n=== RSI STATISTICS ===")
        print(f"Average Entry Spot RSI: {avg_entry_spot_rsi:.2f}")
        print(f"Average Exit Spot RSI: {avg_exit_spot_rsi:.2f}")
        
        # Analyze performance by RSI ranges
        results_df['rsi_range'] = pd.cut(results_df['entry_spot_rsi'], 
                                        bins=[0, 30, 70, 100], 
                                        labels=['Oversold', 'Neutral', 'Overbought'])
        rsi_performance = results_df.groupby('rsi_range')['total_profit'].agg(['mean', 'count'])
        print("\nPerformance by Entry RSI Range:")
        print(rsi_performance)
    
    print("\n✅ All trades completed. Results saved to 'funding_arbitrage_results_2.csv'.")
    return sharpe_ratio, cagr, volatility, max_dd

### Main Function

In [17]:
# Function to run the funding arbitrage bot with risk management and RSI
def funding_arbitrage_bot(df, trade_size, min_funding_rate, max_funding_rate, max_open_positions, stop_loss_pct, max_drawdown_pct, rsi_period, rsi_oversold, rsi_overbought, risk_free_rate):
    global account_balance, open_positions, total_profit, max_balance, daily_returns
    
    results = []
    position_id = 0
    
    # Calculate RSI for both spot and futures prices
    df['spot_rsi'] = calculate_rsi(df['close_spot'], rsi_period)
    df['futures_rsi'] = calculate_rsi(df['close_futures'], rsi_period)
    
    # Add date column for grouping returns by day
    df['date'] = pd.to_datetime(df['timestamp']).dt.date
    
    # Create a daily balance tracker for Sharpe ratio calculation
    daily_balance = {}
    prev_balance = account_balance
    current_date = None
    
    # Filter for arbitrage opportunities
    arbitrage_df = check_arbitrage_opportunity(df, min_funding_rate, max_funding_rate)

    for index, row in df.iterrows():
        # Skip first RSI_PERIOD rows as RSI is not calculated yet
        if index < rsi_period:
            continue
            
        timestamp = row['timestamp']
        current_date = row['date']
        current_spot = row['close_spot']
        current_futures = row['close_futures']
        spot_rsi = row['spot_rsi']
        futures_rsi = row['futures_rsi']
        
        # 1. Check existing positions for stop loss or closing
        positions_to_close = []
        for pos in open_positions:
            # Check if stop loss is triggered
            if check_stop_loss(pos, current_spot, current_futures, stop_loss_pct):
                print(f"\n[Position {pos['id']}] ⚠️ Stop loss triggered! Closing position.")
                positions_to_close.append(pos)
            
            # Check if position has reached its target close time
            elif index >= pos['target_close_index']:
                print(f"\n[Position {pos['id']}] Target close time reached. Closing position.")
                positions_to_close.append(pos)
            
            # Check if RSI signals position exit
            elif (pos['funding_rate'] > 0 and spot_rsi > rsi_overbought) or (pos['funding_rate'] < 0 and spot_rsi < rsi_oversold):
                print(f"\n[Position {pos['id']}] ⚠️ RSI exit signal! Spot RSI: {spot_rsi:.2f}. Closing position.")
                positions_to_close.append(pos)
        
        # Close positions that need closing
        for pos in positions_to_close:
            close_positions(pos['trade_size'], pos['funding_rate'], current_spot)
            
            # Calculate final profit
            profit, funding_profit, spot_pnl, futures_pnl = calculate_total_profit(
                pos['trade_size'], pos['funding_rate'], pos['mark_price'],
                pos['entry_spot'], current_spot, pos['entry_futures'], current_futures
            )
            
            # Update account balance
            account_balance += profit
            total_profit += profit
            max_balance = max(max_balance, account_balance)
            
            # Log results
            results.append([
                timestamp, pos['funding_rate'], pos['trade_size'], 
                pos['entry_spot'], current_spot, pos['entry_futures'], current_futures,
                profit, funding_profit, spot_pnl, futures_pnl, 
                pos['entry_spot_rsi'], spot_rsi, pos['entry_futures_rsi'], futures_rsi
            ])
            
            print(f"Position closed. Profit: ${profit:.2f} (Funding: ${funding_profit:.2f}, Spot: ${spot_pnl:.2f}, Futures: ${futures_pnl:.2f})")
            print(f"RSI at entry - Spot: {pos['entry_spot_rsi']:.2f}, Futures: {pos['entry_futures_rsi']:.2f}")
            print(f"RSI at exit - Spot: {spot_rsi:.2f}, Futures: {futures_rsi:.2f}")
            open_positions.remove(pos)
        
        # 2. Check drawdown - pause trading if exceeded
        current_drawdown = (max_balance - account_balance) / max_balance if max_balance > 0 else 0
        if current_drawdown > max_drawdown_pct:
            print(f"⚠️ Max drawdown exceeded ({current_drawdown:.2%}). Pausing new trades.")
            continue
        
        # 3. Check for new arbitrage opportunity
        funding_rate = row['funding_rate']
        if abs(funding_rate) >= min_funding_rate and abs(funding_rate) <= max_funding_rate:
            
            # Check if we can open new positions
            if len(open_positions) < max_open_positions:
                mark_price = row['close_futures']  # Mark price is usually close to futures price
                entry_spot = row['close_spot']
                entry_futures = row['close_futures']
                
                # Size position based on funding rate magnitude
                adjusted_size = calculate_position_size(funding_rate, trade_size, max_trade_size)
                
                # Calculate capital needed
                position_value = adjusted_size * mark_price
                
                # RSI filter logic for entry
                rsi_entry_condition = False
                
                if funding_rate > 0:  # We'll go long spot, short futures
                    # For long spot: Enter when spot is oversold (low RSI)
                    # For short futures: Enter when futures is overbought (high RSI)
                    rsi_entry_condition = (spot_rsi < rsi_oversold or futures_rsi > rsi_overbought)
                else:  # We'll short spot, long futures
                    # For short spot: Enter when spot is overbought (high RSI)
                    # For long futures: Enter when futures is oversold (low RSI)
                    rsi_entry_condition = (spot_rsi > rsi_overbought or futures_rsi < rsi_oversold)
                
                # Check if we have enough capital (using 50% of available balance per trade) and RSI condition is met
                if position_value <= account_balance * 0.5 and rsi_entry_condition:
                    print(f"\n[New Position {position_id}] Arbitrage Opportunity Found! Funding Rate: {funding_rate}")
                    print(f"RSI - Spot: {spot_rsi:.2f}, Futures: {futures_rsi:.2f}")
                    
                    # Open hedge positions
                    side_spot, side_futures = open_hedge_position(adjusted_size, funding_rate, mark_price)
                    
                    # Target closing at next funding time (8 hours later) or 3 data points if using hourly data
                    target_close_index = index + 3
                    
                    # Record position
                    position = {
                        'id': position_id,
                        'timestamp': timestamp,
                        'funding_rate': funding_rate,
                        'trade_size': adjusted_size,
                        'entry_spot': entry_spot,
                        'entry_futures': entry_futures,
                        'mark_price': mark_price,
                        'open_index': index,
                        'target_close_index': target_close_index,
                        'entry_spot_rsi': spot_rsi,
                        'entry_futures_rsi': futures_rsi
                    }
                    open_positions.append(position)
                    position_id += 1
                elif not rsi_entry_condition:
                    print(f"RSI conditions not met for entry. Spot RSI: {spot_rsi:.2f}, Futures RSI: {futures_rsi:.2f}")
                else:
                    print(f"Insufficient balance for new position. Required: ${position_value:.2f}, Available: ${account_balance:.2f}")
        
        # Track daily balance for returns calculation
        if current_date not in daily_balance:
            # If this is a new day, calculate return from previous day
            if prev_balance > 0 and len(daily_balance) > 0:
                # Calculate daily return
                daily_return = (account_balance / prev_balance) - 1
                daily_returns.append(daily_return)
                
            # Store today's balance
            daily_balance[current_date] = account_balance
            prev_balance = account_balance
    
    # Close any remaining open positions at the end of the backtest
    for pos in open_positions:
        current_spot = df.iloc[-1]['close_spot']
        current_futures = df.iloc[-1]['close_futures']
        spot_rsi = df.iloc[-1]['spot_rsi']
        futures_rsi = df.iloc[-1]['futures_rsi']
        
        close_positions(pos['trade_size'], pos['funding_rate'], current_spot)
        
        # Calculate final profit
        profit, funding_profit, spot_pnl, futures_pnl = calculate_total_profit(
            pos['trade_size'], pos['funding_rate'], pos['mark_price'],
            pos['entry_spot'], current_spot, pos['entry_futures'], current_futures
        )
        
        # Update account balance
        account_balance += profit
        total_profit += profit
        
        # Log results
        results.append([
            df.iloc[-1]['timestamp'], pos['funding_rate'], pos['trade_size'], 
            pos['entry_spot'], current_spot, pos['entry_futures'], current_futures,
            profit, funding_profit, spot_pnl, futures_pnl,
            pos['entry_spot_rsi'], spot_rsi, pos['entry_futures_rsi'], futures_rsi
        ])

    # Final balance for the last day
    if current_date and prev_balance > 0:
        daily_return = (account_balance / prev_balance) - 1
        daily_returns.append(daily_return)

    # Calculate Sharpe ratio
    # Trading days per year depends on your data frequency - for 8h data, it would be 3 times per day * 365
    trading_periods_per_year = (24/8) * 365  # For 8-hour data
    sharpe_ratio = calculate_sharpe_ratio(daily_returns, risk_free_rate, trading_periods_per_year)

    # Save results to CSV
    results_df = pd.DataFrame(results, columns=[
        'timestamp', 'funding_rate', 'trade_size', 'entry_spot', 'close_spot', 
        'entry_futures', 'close_futures', 'total_profit', 'funding_profit', 
        'spot_pnl', 'futures_pnl', 'entry_spot_rsi', 'exit_spot_rsi', 
        'entry_futures_rsi', 'exit_futures_rsi'
    ])
    results_df.to_csv("dataset/funding_arbitrage_results_2.csv", index=False)
    
    # Save daily returns for further analysis
    daily_returns_df = pd.DataFrame({
        'date': list(daily_balance.keys()),
        'balance': list(daily_balance.values())
    })
    daily_returns_df['daily_return'] = daily_returns_df['balance'].pct_change()
    daily_returns_df.to_csv("dataset/daily_returns.csv", index=False)
    
    # Calculate additional performance metrics
    total_trades = len(results_df)
    profitable_trades = len(results_df[results_df['total_profit'] > 0])
    win_rate = profitable_trades / total_trades if total_trades > 0 else 0
    avg_profit = results_df['total_profit'].mean() if total_trades > 0 else 0
    
    # Calculate volatility (standard deviation of returns)
    volatility = np.std(daily_returns, ddof=1) * np.sqrt(trading_periods_per_year) if daily_returns else 0
    
    # Calculate max drawdown
    cumulative_returns = np.cumprod(np.array([1 + r for r in daily_returns]))
    running_max = np.maximum.accumulate(cumulative_returns)
    drawdowns = (cumulative_returns / running_max) - 1
    max_dd = abs(np.min(drawdowns)) if len(drawdowns) > 0 else 0
    
    # Calculate annualized return
    if len(daily_returns) > 0:
        cagr = ((account_balance / 10000) ** (365 / len(daily_balance))) - 1
    else:
        cagr = 0
    
    print("\n=== PERFORMANCE SUMMARY ===")
    print(f"Starting Balance: ${10000:.2f}")
    print(f"Final Balance: ${account_balance:.2f}")
    print(f"Total Profit: ${total_profit:.2f} ({(account_balance/10000 - 1)*100:.2f}%)")
    print(f"Trading Days: {len(daily_balance)}")
    print(f"Annualized Return: {cagr:.2%}")
    print(f"Annualized Volatility: {volatility:.2%}")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"Max Drawdown: {max_dd:.2%}")
    print(f"Total Trades: {total_trades}")
    print(f"Win Rate: {win_rate:.2%}")
    print(f"Average Profit per Trade: ${avg_profit:.2f}")
    
    # RSI effectiveness statistics
    if total_trades > 0:
        avg_entry_spot_rsi = results_df['entry_spot_rsi'].mean()
        avg_exit_spot_rsi = results_df['exit_spot_rsi'].mean()
        print(f"\n=== RSI STATISTICS ===")
        print(f"Average Entry Spot RSI: {avg_entry_spot_rsi:.2f}")
        print(f"Average Exit Spot RSI: {avg_exit_spot_rsi:.2f}")
        
        # Analyze performance by RSI ranges
        results_df['rsi_range'] = pd.cut(results_df['entry_spot_rsi'], 
                                        bins=[0, 30, 70, 100], 
                                        labels=['Oversold', 'Neutral', 'Overbought'])
        rsi_performance = results_df.groupby('rsi_range')['total_profit'].agg(['mean', 'count'])
        print("\nPerformance by Entry RSI Range:")
        print(rsi_performance)
    
    print("\n✅ All trades completed. Results saved to 'funding_arbitrage_results_2.csv'.")
    return sharpe_ratio, cagr, volatility, max_dd

### Run the Main Function

In [19]:
# Run the bot and get performance metrics
sharpe_ratio, cagr, volatility, max_drawdown = funding_arbitrage_bot(
    df, 
    trade_size=trade_size, 
    min_funding_rate=min_funding_rate,
    max_funding_rate=max_funding_rate,
    max_open_positions=max_open_positions,
    stop_loss_pct=stop_loss_pct,
    max_drawdown_pct=max_drawdown_pct,
    rsi_period=rsi_period,
    rsi_oversold=rsi_oversold,
    rsi_overbought=rsi_overbought,
    risk_free_rate=risk_free_rate
)

RSI conditions not met for entry. Spot RSI: 58.43, Futures RSI: 58.52
RSI conditions not met for entry. Spot RSI: 61.99, Futures RSI: 62.08

[New Position 0] Arbitrage Opportunity Found! Funding Rate: 0.00016041
RSI - Spot: 75.86, Futures: 76.22
Opened hedge positions: sell futures and buy spot, Size: 0.01604 BTC ($829.33)

[Position 0] ⚠️ RSI exit signal! Spot RSI: 71.50. Closing position.
Closed hedge positions: sell spot and buy futures, Size: 0.01604 BTC ($830.45)
Position closed. Profit: $3.47 (Funding: $0.13, Spot: $1.65, Futures: $1.69)
RSI at entry - Spot: 75.86, Futures: 76.22
RSI at exit - Spot: 71.50, Futures: 72.06

[New Position 1] Arbitrage Opportunity Found! Funding Rate: 0.0001724
RSI - Spot: 71.50, Futures: 72.06
Opened hedge positions: sell futures and buy spot, Size: 0.01724 BTC ($893.19)

[Position 1] ⚠️ RSI exit signal! Spot RSI: 72.77. Closing position.
Closed hedge positions: sell spot and buy futures, Size: 0.01724 BTC ($898.85)
Position closed. Profit: $12.75 (

C:\Users\julia\AppData\Local\Temp\ipykernel_27612\833251948.py:259: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rsi_performance = results_df.groupby('rsi_range')['total_profit'].agg(['mean', 'count'])


### Export Results

In [13]:
results_df = pd.read_csv("dataset/funding_arbitrage_results_2.csv")
results_df.head()

,timestamp,funding_rate,trade_size,entry_spot,close_spot,entry_futures,close_futures,total_profit,funding_profit,spot_pnl,futures_pnl,entry_spot_rsi,exit_spot_rsi,entry_futures_rsi,exit_futures_rsi
0,2024-02-18 08:00:00,0.000160,0.01604,51671.10,51773.99,51703.7,51808.9,3.470796,0.133032,1.650356,1.687408,75.856108,71.504841,76.217302,72.055292
1,2024-02-18 16:00:00,0.000172,0.01724,51773.99,52137.67,51808.9,52175.9,12.750908,0.153985,6.269843,6.327080,71.504841,72.771345,72.055292,73.274089
2,2024-02-19 16:00:00,0.000153,0.01530,52137.67,51774.73,52175.9,51813.8,-10.970934,0.122178,-5.552982,-5.540130,72.771345,49.976192,73.274089,50.147416
3,2024-02-27 00:00:00,0.000216,0.02156,54476.47,56059.08,54511.6,56100.0,68.620446,0.253471,34.121072,34.245904,71.792184,79.971432,71.654251,79.883674
4,2024-02-27 08:00:00,0.000272,0.02722,56059.08,56745.66,56100.0,56775.0,37.477975,0.415768,18.688708,18.373500,79.971432,84.948793,79.883674,84.805316


In [14]:
results_df['total_profit'].sum()

np.float64(2359.358155310999)